In [19]:
import pandas as pd
from pprint import pprint
data = pd.read_csv("data.csv", index_col=0)

data.columns = data.columns.str.lower().str.replace(" ", "_")

bad_cols = ["club_logo", "photo", "flag", "loaned_from"]

data = data.drop(bad_cols, axis = 1)
data = data.query("position != 'GK'")
data = data[data.overall >=70]
gk_cols = [i for i in data.columns if "gk" in i]
data = data.drop(gk_cols, axis = 1)
data = data.set_index("id")
attribute_cols = data.columns[48:-1].tolist()
features = data[attribute_cols].copy()
features.dropna(inplace=True)
features.shape

(5138, 29)

In [25]:
from sklearn.cluster import SpectralClustering, DBSCAN
from sklearn.preprocessing import StandardScaler

In [26]:
scale = StandardScaler()
X = scale.fit_transform(features)
X = pd.DataFrame(X, columns=features.columns, index=features.index)

In [53]:
model = DBSCAN(eps = 2.3)
model.fit(X)

labels = model.labels_

pd.value_counts(labels)

-1    4113
 0     996
 3      11
 1       8
 4       6
 2       4
dtype: int64

In [37]:
players = pd.DataFrame(data=labels, index=features.index,columns=["labs"])

In [50]:
data.loc[players[players.labs >0].index.tolist()]

,name,age,nationality,overall,potential,club,value,wage,special,preferred_foot,...,aggression,interceptions,positioning,vision,penalties,composure,marking,standingtackle,slidingtackle,release_clause
id,,,,,,,,,,,,,,,,,,,,,
200145,Casemiro,26,Brazil,88,90,Real Madrid,€59.5M,€285K,2170,Right,...,87.0,87.0,69.0,77.0,66.0,84.0,88.0,90.0,87.0,€126.4M
197781,Isco,26,Spain,88,91,Real Madrid,€73.5M,€315K,2137,Right,...,58.0,64.0,78.0,89.0,76.0,86.0,60.0,64.0,51.0,€156.2M
164240,Thiago Silva,33,Brazil,88,88,Paris Saint-Germain,€24M,€165K,2077,Right,...,76.0,89.0,59.0,72.0,60.0,81.0,88.0,89.0,85.0,€44.4M
205600,S. Umtiti,24,France,87,92,FC Barcelona,€57M,€205K,2147,Left,...,81.0,87.0,58.0,58.0,61.0,82.0,90.0,89.0,86.0,€121.1M
172871,J. Vertonghen,31,Belgium,87,87,Tottenham Hotspur,€34M,€155K,2079,Left,...,84.0,89.0,60.0,68.0,66.0,83.0,90.0,87.0,88.0,€62.9M
171877,M. Hamšík,30,Slovakia,87,87,Napoli,€46.5M,€125K,2188,Right,...,56.0,72.0,88.0,86.0,70.0,84.0,75.0,73.0,62.0,€79.1M
218667,Bernardo Silva,23,Portugal,86,91,Manchester City,€59.5M,€180K,2039,Left,...,58.0,49.0,83.0,86.0,68.0,85.0,49.0,44.0,35.0,€114.5M
191043,Alex Sandro,27,Brazil,86,86,Juventus,€36.5M,€160K,2198,Left,...,82.0,82.0,70.0,70.0,60.0,82.0,81.0,84.0,84.0,€60.2M
189596,T. Müller,28,Germany,86,86,FC Bayern München,€45M,€135K,2097,Right,...,57.0,58.0,92.0,85.0,60.0,83.0,44.0,41.0,44.0,€74.3M


In [39]:
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM

In [40]:
iso = IsolationForest()

In [41]:
iso.fit(X)

/Users/georgemcintire/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:214: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/Users/georgemcintire/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:224: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)


IsolationForest(behaviour='old', bootstrap=False, contamination='legacy',
        max_features=1.0, max_samples='auto', n_estimators=100,
        n_jobs=None, random_state=None, verbose=0)

In [42]:
preds = iso.predict(X)

In [43]:
pd.value_counts(preds)

 1    4624
-1     514
dtype: int64

In [44]:
labels = iso.predict(X)

In [45]:
players = pd.DataFrame(data=labels, index=features.index,columns=["labs"])

In [51]:
data.loc[players[players.labs == -1].index.tolist()]

,name,age,nationality,overall,potential,club,value,wage,special,preferred_foot,...,aggression,interceptions,positioning,vision,penalties,composure,marking,standingtackle,slidingtackle,release_clause
id,,,,,,,,,,,,,,,,,,,,,
158023,L. Messi,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,2202,Left,...,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,€226.5M
20801,Cristiano Ronaldo,33,Portugal,94,94,Juventus,€77M,€405K,2228,Right,...,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,€127.1M
190871,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,2143,Right,...,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,€228.1M
192985,K. De Bruyne,27,Belgium,91,92,Manchester City,€102M,€355K,2281,Right,...,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,€196.4M
183277,E. Hazard,27,Belgium,91,91,Chelsea,€93M,€340K,2142,Right,...,54.0,41.0,87.0,89.0,86.0,91.0,34.0,27.0,22.0,€172.1M
177003,L. Modrić,32,Croatia,91,91,Real Madrid,€67M,€420K,2280,Right,...,62.0,83.0,79.0,92.0,82.0,84.0,60.0,76.0,73.0,€137.4M
176580,L. Suárez,31,Uruguay,91,91,FC Barcelona,€80M,€455K,2346,Right,...,87.0,41.0,92.0,84.0,85.0,85.0,62.0,45.0,38.0,€164M
155862,Sergio Ramos,32,Spain,91,91,Real Madrid,€51M,€380K,2201,Right,...,88.0,90.0,60.0,63.0,75.0,82.0,87.0,92.0,91.0,€104.6M
188545,R. Lewandowski,29,Poland,90,90,FC Bayern München,€77M,€205K,2152,Right,...,80.0,39.0,91.0,77.0,88.0,86.0,34.0,42.0,19.0,€127.1M


In [54]:
features.groupby(labels).mean()

,crossing,finishing,headingaccuracy,shortpassing,volleys,dribbling,curve,fkaccuracy,longpassing,ballcontrol,...,longshots,aggression,interceptions,positioning,vision,penalties,composure,marking,standingtackle,slidingtackle
-1,60.269633,56.070022,63.517384,69.926331,53.490883,67.000243,58.542183,53.044493,63.281789,70.636761,...,59.235108,66.301726,56.176513,61.093363,62.268174,57.585218,69.407002,56.141746,56.385363,53.212983
0,68.298193,59.728916,61.875502,72.925703,59.002008,71.878514,66.522088,60.252008,68.479920,73.442771,...,65.895582,69.375502,62.647590,66.885542,68.107430,60.207831,70.722892,60.991968,62.693775,60.002008
1,74.500000,76.250000,59.750000,75.375000,72.875000,83.000000,76.500000,72.750000,67.125000,81.000000,...,75.500000,65.000000,40.750000,77.500000,75.250000,68.000000,76.500000,39.125000,34.875000,30.125000
2,72.000000,41.500000,72.500000,72.500000,39.250000,67.250000,59.250000,38.000000,62.000000,70.750000,...,41.000000,75.000000,74.000000,62.000000,56.750000,43.750000,71.250000,73.750000,77.750000,78.500000
3,72.000000,68.909091,46.454545,74.090909,61.818182,77.727273,73.636364,71.545455,70.727273,76.636364,...,68.818182,44.181818,35.272727,68.727273,71.454545,65.000000,63.727273,38.636364,31.090909,29.727273
4,63.500000,53.166667,47.500000,74.666667,62.500000,68.333333,70.833333,68.166667,72.000000,71.666667,...,69.666667,71.333333,65.333333,63.500000,71.333333,65.333333,71.500000,65.000000,64.500000,62.333333


In [55]:
features.corr()

,crossing,finishing,headingaccuracy,shortpassing,volleys,dribbling,curve,fkaccuracy,longpassing,ballcontrol,...,longshots,aggression,interceptions,positioning,vision,penalties,composure,marking,standingtackle,slidingtackle
crossing,1.000000,0.410274,-0.428622,0.571281,0.443952,0.693842,0.720102,0.581995,0.473142,0.648930,...,0.547632,-0.190285,-0.080073,0.576572,0.639076,0.358704,0.277992,-0.113611,-0.098572,-0.087134
finishing,0.410274,1.000000,-0.198182,0.335861,0.815739,0.703791,0.597282,0.531347,0.042088,0.649077,...,0.769600,-0.397132,-0.644311,0.841452,0.635287,0.708555,0.308387,-0.643078,-0.673966,-0.691927
headingaccuracy,-0.428622,-0.198182,1.000000,-0.244635,-0.155678,-0.430313,-0.393391,-0.359955,-0.245397,-0.318660,...,-0.282956,0.399070,0.202946,-0.253374,-0.397389,-0.105737,0.061235,0.225517,0.209345,0.203276
shortpassing,0.571281,0.335861,-0.244635,1.000000,0.378905,0.583377,0.546072,0.504274,0.732941,0.723798,...,0.500692,0.000523,0.143819,0.437431,0.707160,0.316962,0.497086,0.101586,0.116331,0.086723
volleys,0.443952,0.815739,-0.155678,0.378905,1.000000,0.656849,0.649220,0.583481,0.120488,0.637112,...,0.755780,-0.299172,-0.523449,0.756945,0.622410,0.706037,0.386060,-0.527886,-0.559463,-0.575290
dribbling,0.693842,0.703791,-0.430313,0.583377,0.656849,1.000000,0.717811,0.554564,0.317868,0.859073,...,0.705366,-0.345904,-0.397614,0.788453,0.751295,0.518889,0.363417,-0.409270,-0.416416,-0.420693
curve,0.720102,0.597282,-0.393391,0.546072,0.649220,0.717811,1.000000,0.765373,0.391809,0.701402,...,0.712621,-0.247955,-0.266631,0.668210,0.706328,0.578401,0.369523,-0.296117,-0.298259,-0.306710
fkaccuracy,0.581995,0.531347,-0.359955,0.504274,0.583481,0.554564,0.765373,1.000000,0.429108,0.572651,...,0.689491,-0.200936,-0.207369,0.531751,0.646209,0.602851,0.325467,-0.245330,-0.242235,-0.259101
longpassing,0.473142,0.042088,-0.245397,0.732941,0.120488,0.317868,0.391809,0.429108,1.000000,0.426293,...,0.318298,0.122969,0.399576,0.123435,0.537960,0.109890,0.361964,0.335654,0.372159,0.348222
ballcontrol,0.648930,0.649077,-0.318660,0.723798,0.637112,0.859073,0.701402,0.572651,0.426293,1.000000,...,0.680236,-0.247984,-0.268708,0.730263,0.766669,0.521144,0.489744,-0.286369,-0.297673,-0.314105


In [56]:
features_matrix = features.values

In [57]:
corr_matrix = features.corr().abs()

In [59]:
import numpy as np

In [60]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

In [63]:
to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]
to_drop

['volleys',
 'ballcontrol',
 'sprintspeed',
 'balance',
 'positioning',
 'marking',
 'standingtackle',
 'slidingtackle']

In [66]:
from sklearn.cluster import KMeans

In [67]:
km = KMeans(3)

In [68]:
km.fit(corr_matrix)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [70]:
corr_matrix.groupby(km.labels_).mean()

,crossing,finishing,headingaccuracy,shortpassing,volleys,dribbling,curve,fkaccuracy,longpassing,ballcontrol,...,longshots,aggression,interceptions,positioning,vision,penalties,composure,marking,standingtackle,slidingtackle
0,0.583431,0.653582,0.299435,0.523071,0.653105,0.699502,0.678932,0.610280,0.314673,0.687763,...,0.696173,0.245967,0.341353,0.687046,0.679861,0.575031,0.360769,0.355263,0.366970,0.374977
1,0.113935,0.610083,0.248031,0.089797,0.497052,0.397980,0.283134,0.230994,0.315716,0.282968,...,0.370822,0.678206,0.866097,0.498533,0.251626,0.433198,0.038461,0.839472,0.880695,0.874151
2,0.376821,0.240127,0.387817,0.327738,0.234988,0.406951,0.342039,0.275847,0.277660,0.379475,...,0.284085,0.269180,0.236670,0.305611,0.362888,0.175865,0.235536,0.225841,0.228431,0.215913


In [72]:
corr_matrix.sum()

crossing           12.505927
finishing          14.601837
headingaccuracy     9.310412
shortpassing       11.049355
volleys            13.978611
dribbling          15.852444
curve              14.341114
fkaccuracy         12.457365
longpassing         8.760599
ballcontrol        14.838268
acceleration       11.220679
sprintspeed         9.589129
agility            13.584967
reactions           7.571344
balance            11.892793
shotpower          11.329882
jumping             4.866489
stamina             6.162928
strength           10.268294
longshots          14.441378
aggression          9.526376
interceptions      11.476130
positioning        15.167423
vision             14.405052
penalties          11.975072
composure           7.598440
marking            11.429456
standingtackle     11.825371
slidingtackle      11.779553
dtype: float64

In [74]:
# corr_matrix["positioning"]